# Model Training for Iris Classification using Azure AutoML

In this notebook, we'll use Azure AutoML to train a model for Iris flower classification. We'll perform the following steps:
1. Connect to the Azure ML workspace
2. Load the prepared dataset
3. Configure AutoML
4. Run AutoML
5. Examine the results
6. Retrieve the best model

## 1. Connect to the Azure ML workspace and load necessary libraries

In [ ]:
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import logging

# Connect to your workspace
ws = Workspace.from_config()
print(f"Connected to workspace: {ws.name}")

## 2. Load the prepared dataset

In [ ]:
# Get the dataset
dataset = Dataset.get_by_name(ws, name='iris_dataset')
df = dataset.to_pandas_dataframe()

# Split features and target
X = df.drop(['species'], axis=1)
y = df['species']

print(f"Dataset shape: {df.shape}")
print(f"Features: {X.columns.tolist()}")
print(f"Target: {y.name}")

## 3. Configure AutoML

In [ ]:
# Set parameters for AutoML run
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric": 'accuracy'
}

automl_config = AutoMLConfig(task='classification',
                             training_data=dataset,
                             label_column_name='species',
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automl_errors.log',
                             **automl_settings)

## 4. Run AutoML

In [ ]:
# Create an experiment
experiment = Experiment(ws, "iris_classification_automl")

# Submit the experiment
automl_run = experiment.submit(automl_config, show_output=True)

## 5. Examine the results

In [ ]:
# Get the best model
best_run, fitted_model = automl_run.get_output()
print(f"Best run id: {best_run.id}")
print(f"Best run algorithm: {best_run.properties['run_algorithm']}")
print(f"Best run accuracy: {best_run.properties['accuracy']}")

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    print(f"{metric_name}: {best_run_metrics[metric_name]}")

# Get feature importances
if hasattr(fitted_model, 'feature_importances_'):
    importances = fitted_model.feature_importances_
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    print("\nFeature Importances:")
    print(feature_importance)

## 6. Retrieve and save the best model

In [ ]:
# Register the model
model_name = "iris_classification_model"
model_description = "AutoML model for Iris classification"
best_run.register_model(model_name=model_name, description=model_description)

print(f"Model registered: {model_name}")

## Conclusion

We've successfully used Azure AutoML to train a model for Iris flower classification. The best model has been registered in our Azure ML workspace and is now ready for deployment in our next notebook.